# Capitulo 9: Clases imbalanceadas

# BIbliotecas utilizadas

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from itertools import zip_longest
from tabulate import tabulate
import pdfkit
import seaborn as sns
from itertools import zip_longest
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, cross_val_score  
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import sklearn.linear_model as lm
from sklearn.linear_model import LogisticRegression
import sklearn.tree as tree
from sklearn.metrics import accuracy_score
import sklearn.neighbors as neighbors
import sklearn.naive_bayes as nb
import sklearn.svm as svm
import sklearn.ensemble as ensemble
from sklearn.metrics import confusion_matrix
from yellowbrick.classifier import ConfusionMatrix
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.features import (JointPlotVisualizer,)
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost

*Si está clasificando datos y las clases no están relativamente equilibradas en tamaño, el sesgo hacia clases más populares puede llevar en su modelo. Por ejemplo, si tienes 1 caso positivo y 99 casos negativos, puede obtener una precisión del 99% simplemente clasificando todo como negativo. Hay varias opciones para abordar las clases desequilibradas.*

# Utilice la métrica diferente

Una sugerencia es utilizar una medida distinta a la precisión (AUC es una buena elección) para calibrar modelos. La precisión y la recuperación son También mejores opciones cuando los tamaños de destino son diferentes. Sin embargo,También hay otras opciones a considerar.

# Algoritmos y conjuntos basados ​​en árboles

Los modelos basados ​​en árboles pueden funcionar mejor según la distribución de la clase más pequeña. Si tienden a estar agrupados,se puede clasificar más fácilmente. Los métodos conjuntos pueden ayudar aún más a sacar a la minoría clases. El embolsado y el impulso son opciones que se encuentran en los modelos de árbol.como bosques aleatorios y aumento de gradiente extremo (XGBoost).

# Penalize Models

*Muchos modelos de clasificación de scikit-learn respaldan la parámetro class_weight. Configurarlo en 'equilibrado' intentará regularizar las clases minoritarias e incentivar el modelo para clasificar ellos correctamente. Alternativamente, puede buscar en la cuadrícula y especificar las opciones de peso pasando una clase de mapeo de diccionario al peso (dar mayor peso a las clases más pequeñas). La biblioteca XGBoost tiene el parámetro max_delta_step, que Se puede configurar de 1 a 10 para que el paso de actualización sea más conservador.También tiene el parámetro scale_pos_weight que establece el proporción de muestras negativas a positivas (para clases binarias). También, eval_metric debe establecerse en 'auc' en lugar del valor predeterminado valor de 'error' para la clasificación. El modelo KNN tiene un parámetro de ponderación que puede sesgar a los vecinos. que están más cerca. Si las muestras de clases minoritarias están cerca juntos, establecer este parámetro en 'distancia' puede mejorar actuación.*

# Importacion de los datos

In [21]:
# Ruta completa al archivo CSV
file_path = 'C:/Users/felip/OneDrive/Documentos/Python/Codigos/Ciencia_Datos/Datos_titanic_CSV/titanic3.csv'
# Importar la base de datos
df = pd.read_csv(file_path)
X= df
# Puedes imprimir el DataFrame si lo deseas
# print(df)

# Limpieza de los datos 

In [22]:
def tweak_titanic(df):
    df = df.drop(columns=[
        "name",
        "ticket",
        "home.dest",
        "boat",
        "body",
        "cabin"
    ]).pipe(pd.get_dummies, drop_first=True)
    return df

ti_df = tweak_titanic(df)
#print(ti_df)
def get_train_test_X_y(df, y_col, size=0.3, std_cols=None):
    #Obtencion del conjunto de prueba y entreanamiento
    y = df[y_col]
    X = df.drop(columns=y_col)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=size, random_state=42)
    cols = X.columns
    num_cols = ["pclass", "age", "sibsp", "parch", "fare"]
    #Imputacion de los valores faltantes 
    fi = impute.IterativeImputer()
    X_train.loc[:, num_cols] = fi.fit_transform(X_train[num_cols])
    X_test.loc[:, num_cols] = fi.transform(X_test[num_cols])
    #Estandarizacion de los valores 
    if std_cols:
        std = preprocessing.StandardScaler()
        X_train.loc[:, std_cols] = std.fit_transform(X_train[std_cols])
        X_test.loc[:, std_cols] = std.transform(X_test[std_cols])
    return X_train, X_test, y_train, y_test

ti_df = tweak_titanic(df)
std_cols = "pclass,age,sibsp,fare".split(",")
X_train, X_test, y_train, y_test = get_train_test_X_y(ti_df, "survived", std_cols=std_cols)

# Minoría de muestreo superior

In [23]:
from sklearn.utils import resample
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]
df_upsample = resample(
    surv_df,
    replace=True,
    n_samples=len(death_df),
    random_state=42,
    )

In [24]:
df2 = pd.concat([death_df, df_upsample])

In [25]:
df2.survived.value_counts()

survived
0    809
1    809
Name: count, dtype: int64

*También podemos usar la biblioteca de aprendizaje desequilibrado para muestrear aleatoriamente con reemplazo:*

In [29]:
from imblearn.over_sampling import RandomOverSampler
y = df["survived"]
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)
pd.Series(y_ros).value_counts()

survived
1    809
0    809
Name: count, dtype: int64

# Generate Minority Data

*La biblioteca de aprendizaje desequilibrado también puede generar nuevas muestras de clases minoritarias con el sobremuestreo de minorías sintéticas Técnica (SMOTE) y Sintética Adaptativa (ADASYN)algoritmos de enfoque de muestreo. SMOTE funciona eligiendo uno de sus k vecinos más cercanos, conectando una línea a uno de ellos, y eligiendo un punto a lo largo de esa línea. ADASYN es similar a SMOTE, pero genera más muestras de aquellas que están más difícil de aprender. Las clases en imbanced-learn se nombran over_sampling.SMOTE y over_sampling.ADASYN.*

# Downsampling Majority

*Otro método para equilibrar las clases es reducir la muestra de la mayoría. clases. Aquí hay un ejemplo de sklearn:*

In [35]:
from sklearn.utils import resample
mask = df.survived == 1
surv_df = df[mask]
death_df = df[~mask]
df_downsample = resample(
    death_df,
    replace=False,
    n_samples=len(surv_df),
    random_state=42,
)

df3 = pd.concat([surv_df, df_downsample])
df3.survived.value_counts()

survived
1    500
0    500
Name: count, dtype: int64

*No utilice el reemplazo al reducir la resolución*

*La biblioteca imbalanced-learn también implementa varios algoritmos de reducción de muestreo:*

- *ClusterCentroids*
  *Esta clase utiliza K-means para sintetizar datos con los centroides.*

- *RandomUnderSampler*
  *Esta clase selecciona muestras al azar.*

- *NearMiss*
  *Esta clase utiliza vecinos más cercanos para reducir el muestreo.*

- *TomekLink*
  *Esta clase reduce el muestreo eliminando muestras que están cerca entre sí.*

- *EditedNearestNeighbours*
  *Esta clase elimina muestras que tienen vecinos que no pertenecen a la mayoría o todos de la misma clase.*

- *RepeatedNearestNeighbours*
  *Esta clase llama repetidamente a EditedNearestNeighbours.*

- *AllKNN*
  *Esta clase es similar, pero aumenta el número de vecinos más cercanos durante las iteraciones de reducción de muestreo.*

- *CondensedNearestNeighbour*
  *Esta clase elige una muestra de la clase a reducir,*
  *luego itera a través de las otras muestras de la clase,*
  *y si KNN no clasifica erróneamente, agrega esa muestra.*

- *OneSidedSelection*
  *Esta clase elimina muestras ruidosas.*

- *NeighbourhoodCleaningRule*
  *Esta clase utiliza los resultados de EditedNearestNeighbours y*
  *aplica KNN a esos resultados.*

- *InstanceHardnessThreshold*
  *Esta clase entrena un modelo y luego elimina muestras con*
  *probabilidades bajas.*

*Todos estos clases admiten el método .fit_sample.*


# Upsampling Then Downsampling

*The imbalanced-learn library implements SMOTEENN and SMOTE Tomek, which both upsample and then apply downsampling to clean up the data.*